In [1]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
print(os.getcwd())

c:\Users\utcpret\Documents\NF26\nf26_project


In [2]:
!pip install geopy


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\utcpret\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, date

import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row
from etl_warehouse import *

C:\Users\utcpret\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
C:\Users\utcpret\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
## Définition du fuseau horaire sur paris ##

spark.conf.set("spark.sql.session.timeZone", "Europe/Paris")


TABLE PERSONNEL
pas besoin de la créer de manière journalière car on pouvait l'extraire directement

In [6]:
#BERLIN
psdf_personnel_BERLIN = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_BERLIN/PERSONNEL_BERLIN.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_BERLIN = psdf_personnel_BERLIN.to_spark(index_col='ID_PERSONNEL')

#LONDON
psdf_personnel_LONDON = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_LONDON/PERSONNEL_LONDON.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_LONDON = psdf_personnel_LONDON.to_spark(index_col='ID_PERSONNEL')

#LOSANGELES
psdf_personnel_LOSANGELES = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_LOSANGELES/PERSONNEL_LOSANGELES.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_LOSANGELES = psdf_personnel_LOSANGELES.to_spark(index_col='ID_PERSONNEL')

#NEWYORK
psdf_personnel_NEWYORK = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_NEWYORK/PERSONNEL_NEWYORK.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_NEWYORK = psdf_personnel_NEWYORK.to_spark(index_col='ID_PERSONNEL')

#PARIS
psdf_personnel_PARIS = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_PARIS/PERSONNEL_PARIS.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_PARIS = psdf_personnel_PARIS.to_spark(index_col='ID_PERSONNEL')

#SHANGHAI
psdf_personnel_SHANGHAI = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_SHANGHAI/PERSONNEL_SHANGHAI.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_SHANGHAI = psdf_personnel_SHANGHAI.to_spark(index_col='ID_PERSONNEL')
 
 
 
# Code corrigé pour la dimension personnel
import os
from pyspark.sql.functions import lit, col

# Créer le répertoire DIM_PERSONNEL s'il n'existe pas
os.makedirs("DATA_WAREHOUSE/DIM_PERSONNEL", exist_ok=True)

# Union de tous les DataFrames du personnel
sdf_personnel_all = sdf_personnel_BERLIN.union(sdf_personnel_LONDON) \
    .union(sdf_personnel_LOSANGELES).union(sdf_personnel_NEWYORK) \
    .union(sdf_personnel_PARIS).union(sdf_personnel_SHANGHAI)

# Ajouter la colonne VILLE pour indiquer la provenance
sdf_personnel_berlin_with_city = sdf_personnel_BERLIN.withColumn("VILLE", lit("BERLIN"))
sdf_personnel_london_with_city = sdf_personnel_LONDON.withColumn("VILLE", lit("LONDON"))
sdf_personnel_la_with_city = sdf_personnel_LOSANGELES.withColumn("VILLE", lit("LOSANGELES"))
sdf_personnel_ny_with_city = sdf_personnel_NEWYORK.withColumn("VILLE", lit("NEWYORK"))
sdf_personnel_paris_with_city = sdf_personnel_PARIS.withColumn("VILLE", lit("PARIS"))
sdf_personnel_shanghai_with_city = sdf_personnel_SHANGHAI.withColumn("VILLE", lit("SHANGHAI"))

# Union de tous les DataFrames avec la colonne VILLE
sdf_personnel_all_with_city = sdf_personnel_berlin_with_city.union(sdf_personnel_london_with_city) \
    .union(sdf_personnel_la_with_city).union(sdf_personnel_ny_with_city) \
    .union(sdf_personnel_paris_with_city).union(sdf_personnel_shanghai_with_city)

# Afficher un échantillon
sdf_personnel_all_with_city.show(7)

# Spécifier le chemin complet du fichier CSV (pas seulement le dossier)
dim_personnel_path = "DATA_WAREHOUSE/DIM_PERSONNEL/dimension_personnel.csv"

# Utiliser pandas pour sauvegarder en un seul fichier
personnel_pandas_df = sdf_personnel_all_with_city.toPandas()
personnel_pandas_df.to_csv(dim_personnel_path, index=False)

print(f"Table dimension PERSONNEL créée avec {sdf_personnel_all_with_city.count()} employés")

+--------------------+-------------+----------------+----------+-----------+-----------+-----------+-----------------+-------------+--------+----------+---------+---------+------+-------+------------------+---------------------+-------------------+
|        ID_PERSONNEL|NOM_PERSONNEL|PRENOM_PERSONNEL|  DT_NAISS|VILLE_NAISS| PAYS_NAISS|   NUM_SECU|IND_PAYS_NUM_TELP|NUM_TELEPHONE|NUM_VOIE|  DSC_VOIE|CMPL_VOIE|CD_POSTAL| VILLE|   PAYS|FONCTION_PERSONNEL|TS_CREATION_PERSONNEL|  TS_MAJ_PPERSONNEL|
+--------------------+-------------+----------------+----------+-----------+-----------+-----------+-----------------+-------------+--------+----------+---------+---------+------+-------+------------------+---------------------+-------------------+
|KeyPers_Berlin_12...|        Name0|       FistName0|1993-11-04|      Osaka|      Japan|NS000000000|             NULL|+336##0263188|      57|NomVoie940|     NULL|    #8830|BERLIN|Germany|    Dateningenieur|  2010-09-09 11:22:17|2010-09-09 11:22:17|
|Key

EXECUTION ETL boucle avec tous les jours de 2024.

In [7]:
# process_full_year()

In [8]:
# import pyspark.pandas as ps
# from pyspark.sql import functions as F
# from pyspark.sql.types import *

# # Charger les tables de dimension et de faits
# dim_personnel = spark.read.csv("DATA_WAREHOUSE/DIM_PERSONNEL/dimension_personnel.csv", header=True, inferSchema=True)
# dim_materiel = spark.read.csv("DATA_WAREHOUSE/DIM_MATERIEL/dimension_materiel.csv", header=True, inferSchema=True) 
# dim_date = spark.read.csv("DATA_WAREHOUSE/DIM_DATE/dimension_date.csv", header=True, inferSchema=True)
# faits_materiel = spark.read.csv("DATA_WAREHOUSE/FAITS_MATERIEL/faits_materiel.csv", header=True, inferSchema=True)

# # Examiner le schéma des tables pour comprendre les colonnes disponibles
# print("Schema de la table PERSONNEL:")
# dim_personnel.printSchema()

# # Créer les vues pour les requêtes SQL
# dim_personnel.createOrReplaceTempView("personnel")
# dim_materiel.createOrReplaceTempView("materiel")
# dim_date.createOrReplaceTempView("dates")
# faits_materiel.createOrReplaceTempView("faits_materiel")

# # Question 1: Combien d'ingénieurs informaticiens travaillent sur le site de Paris?
# q1 = spark.sql("""
#     SELECT COUNT(*) as nb_ingenieurs_info_paris
#     FROM personnel
#     WHERE FONCTION_PERSONNEL = 'Ingénieur Informaticien' 
#     AND VILLE = 'PARIS'
# """)
# q1.show()

# # Question 2: Combien d'ingénieurs Data travaillent sur les sites de London?
# q2 = spark.sql("""
#     SELECT COUNT(*) as nb_ingenieurs_data_london
#     FROM personnel
#     WHERE FONCTION_PERSONNEL = 'Data Engineer' 
#     AND VILLE = 'LONDON'
# """)
# q2.show()

# # Question 3: Combien de cadres travaillent dans l'organisation (tous sites compris)?
# q3 = spark.sql("""
#     SELECT COUNT(*) as nb_cadres_total
#     FROM personnel
#     WHERE FONCTION_PERSONNEL LIKE '%Cadre%'  
#       OR FONCTION_PERSONNEL LIKE '%Führungskraft%' 
#       OR FONCTION_PERSONNEL LIKE '%Business Executive%'
# """)
# q3.show()

# # Question 4: Combien de PC portables ont été achetés par l'organisation entre mai et octobre 2024?
# # D'abord, vérifions le schéma des tables pour cette requête
# print("Schema de la table MATERIEL:")
# dim_materiel.printSchema()
# print("Schema de la table DATES:")
# dim_date.printSchema()
# print("Schema de la table FAITS_MATERIEL:")
# faits_materiel.printSchema()

# q4 = spark.sql("""
#     SELECT COUNT(DISTINCT f.ID_MATERIELINFO) as nb_pc_portables
#     FROM faits_materiel f
#     JOIN materiel m ON f.ID_MATERIELINFO = m.ID_MATERIELINFO
#     JOIN dates d ON f.KeyDate = d.KeyDate
#     WHERE m.TYPE LIKE '%PC%porta%'
#     AND d.MOIS BETWEEN 5 AND 10
#     AND d.ANNEE = 2024
# """)
# q4.show()

In [9]:
date1 = '20240429'
executer_etl_pour_date(date1)
date2 = '20240907'
executer_etl_pour_date(date2)

Démarrage de l'ETL pour la date: 20240429
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_BERLIN\BDD_BGES_BERLIN_MISSION\MISSION_20240429.txt - 27 lignes
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_PARIS\BDD_BGES_PARIS_MISSION\MISSION_20240429.txt - 33 lignes
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_LONDON\BDD_BGES_LONDON_MISSION\MISSION_20240429.txt - 16 lignes


Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_LOSANGELES\BDD_BGES_LOSANGELES_MISSION\MISSION_20240429.txt - 12 lignes
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_NEWYORK\BDD_BGES_NEWYORK_MISSION\MISSION_20240429.txt - 29 lignes
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_SHANGHAI\BDD_BGES_SHANGHAI_MISSION\MISSION_20240429.txt - 15 lignes
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_SHANGHAI\BDD_BGES_SHANGHAI_MISSION\MISSION_20240430.txt - 5 lignes
Table dimension DATE créée avec 1 dates
Geocoder error (attempt 1) for 'Shanghai': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Shanghai&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Erreur lors de la création/mise à jour de la table MISSION: can't multiply sequence by non-int of type 'float'
Traceback (most recent call last):
  File "c:\Users\